In [1]:
from cube import Cube
from tile import Tile
from alt_map import create_hex_map, valid_cubes

In [2]:
import random

In [3]:
random.seed(1945)
n_x = 129
n_y = 65
rgb_from_ijk = {cub.tuple():(random.randint(0,255), random.randint(0,255), random.randint(0,255)) for cub in valid_cubes(n_x,n_y)}

In [4]:
max_x = 10*(n_x*3-3)
max_y = 17*(n_y*2-2)

In [5]:
img = create_hex_map(rgb_from_ijk, max_x,max_y,n_x,n_y)
img.show()

In [6]:
pid_from_cube = {cub.tuple(): ind for ind, cub in enumerate(valid_cubes(n_x,n_y))}
cube_from_pid = {v: k for k, v in pid_from_cube.items()}
weight_from_pid = {pid: random.randint(1,8) for pid in cube_from_pid.keys()}

In [7]:
weight_from_cube = {Cube(cube_from_pid[k]): v for k,v in weight_from_pid.items()}

In [8]:
centers = random.sample(sorted(cube_from_pid.values()),40)
print(centers)
print(sum([sum([x == c for x in centers]) for c in centers]))

[(112, -78, -34), (29, -70, 41), (36, -53, 17), (25, -72, 47), (85, -90, 5), (94, -106, 12), (106, -110, 4), (38, -45, 7), (126, -116, -10), (64, -62, -2), (93, -79, -14), (31, -45, 14), (123, -123, 0), (26, -76, 50), (2, -40, 38), (98, -83, -15), (35, -49, 14), (82, -96, 14), (30, -72, 42), (69, -38, -31), (106, -113, 7), (47, -57, 10), (79, -95, 16), (124, -69, -55), (53, -80, 27), (26, -73, 47), (97, -92, -5), (15, -70, 55), (115, -83, -32), (20, -27, 7), (67, -93, 26), (33, -30, -3), (115, -73, -42), (42, -40, -2), (72, -57, -15), (43, -46, 3), (12, -53, 41), (116, -116, 0), (5, -37, 32), (73, -40, -33)]
40


In [9]:
def voronoi(centers, weight_from_cube):
    """Uses the domain of weight_from_cube to determine which cubes are eligible to be filled.
    - centers is a list of cubes
    - weight_from_cube is a dictionary of cube tuples to numbers
    returns a dictionary of cube tuples to the index of the centers list that they correspond to."""
    # distances = {cub: [0]*len(centers) if }
    centers = [Cube(x) for x in centers]  #If they're tuples, make them cubes.
    # See if any of the centers are on top of each other. TODO: Handle that case instead of just erroring.
    assert len(centers) == sum([sum([x == c for x in centers]) for c in centers])
    # Overall strategy: 
    # - seed all of the centers with distance 0
    # - add all neighbors of the centers to expand sets with the weight of the center as the 'distance'
    # - while there are any elements in the expansion sets, pick the lowest distance one and expand it
    # - if it's the lowest distance to the cell it expands to, add that to the expansion set
    # = finally, point each cell at its lowest distance source. 
    distmap = {Cube(k):{} for k in weight_from_cube.keys()}
    # Each center is in its own voronoi cell
    for cind, center in enumerate(centers):
        distmap[center][cind] = 0
        to_explore = set([center])
        # explored = set()
        while len(to_explore) > 0:
            cub = to_explore.pop()
            base_dist = distmap[cub][cind] + weight_from_cube[cub]
            expanding = [c for c in cub.neighbors() if c in weight_from_cube ]  # and c not in explored]
            for other in expanding:
                if len(distmap[other]) == 0 or min(distmap[other].values()) > base_dist:
                    distmap[other][cind] = base_dist
                    to_explore.add(other)
                else:
                    continue            
    #TODO: This currently is 'kind of fast' but could be faster. Two possible improvements:
    # Instead of starting with center 1 and fully calculating the distances, start with all centers simultaneously.
    # The previous, but instead of having to argmin the to_explore queue each time, have the queue split out by distance so that it's obvious where the next place to go is.
    result = dict()
    for cub in weight_from_cube.keys():
        dists = distmap.get(Cube(cub),{0:0})
        result[cub] = min(dists, key=dists.get)
    return result
        

In [10]:
def iterative_voronoi(num_centers, weight_from_cube, min_size, max_iters=20):
    """Given a set of weights, seed num_centers random centers and then keep going until all of the regions are at least min_size.
    Returns the pair of centers and ind_from_cube mapping."""
    assert num_centers * min_size < len(weight_from_cube)
    # assert num_centers * max_size > len(weight_from_cube)
    centers = random.sample(list(weight_from_cube.keys()),num_centers)
    guess = voronoi(centers, weight_from_cube)
    sizes = {c:0 for c in range(num_centers)}
    means = {c:Cube(0,0,0) for c in range(num_centers)}
    for cub, ind in guess.items():
        sizes[ind] += 1
        means[ind].add_in_place(cub)
    print(sum([min_size <= sizes[ind] for ind in range(num_centers)]))
    if all([min_size <= sizes[ind] for ind in range(num_centers)]):
        return centers, guess
    iter = 1
    while not all([min_size <= sizes[ind] for ind in range(num_centers)]):
        to_remove = []
        for ind in range(len(centers)):
            if min_size <= sizes[ind]:
                x = means[ind].x // sizes[ind]
                y = means[ind].y // sizes[ind]
                z = -x-y
                centers[ind] = Cube(x,y,z)
            elif sizes[ind] < min_size:
                to_remove.append(ind)
        argmaxes = sorted(sizes, key=sizes.get, reverse=True)
        for from_ind, to_ind in zip(to_remove, argmaxes):
            centers[from_ind] = random.sample([k for k,v in guess.items() if v==to_ind and k != centers[to_ind]], k=1)[0]
        guess = voronoi(centers, weight_from_cube)
        iter += 1
        if iter >= max_iters:
            return centers, guess
        sizes = {c:0 for c in range(num_centers)}
        means = {c:Cube(0,0,0) for c in range(num_centers)}
        for cub, ind in guess.items():
            sizes[ind] += 1
            means[ind].add_in_place(cub)
        print(sum([min_size <= sizes[ind] for ind in range(num_centers)]))
    return centers, guess

        



In [23]:
centers, result = iterative_voronoi(num_centers=80, weight_from_cube=weight_from_cube, min_size=61)
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()
img.save("weighted.png")

54
69
73
72
73
73
69
75
72
72
73
73
72
77
78
79
76
76
77


In [20]:
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [17]:
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] if v==1 else (0,0,0) for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [65]:
img = create_hex_map({cub.tuple(): (v[1]//3,0,0) for cub, v in distmap.items()}, max_x,max_y,n_x,n_y)
img.show()

In [62]:
max([max([distmap[c][x] for c in weight_from_cube.keys()]) for x in range(len(centers))])

620

In [49]:
rgb_from_ijk[centers[29]]

(118, 191, 215)

In [26]:
len(weight_from_cube)//61

136

In [13]:
def simple_voronoi(centers, weights_from_cube):
    """Uses the domain of weights_from_cube to determine which cubes are eligible to be filled.
    - centers is a list of cubes
    - weights_from_cube is a dictionary of cube tuples to numbers
    returns a dictionary of cube tuples to the index of the centers list that they correspond to."""
    # distances = {cub: [0]*len(centers) if }
    centers = [Cube(x) for x in centers]  #If they're tuples, make them cubes.
    # See if any of the centers are on top of each other. TODO: Handle that case instead of just erroring.
    assert len(centers) == sum([sum([x == c for x in centers]) for c in centers])
    result = {}
    for cub in weights_from_cube.keys():
        dists = {cind:Cube(cub).dist(c) for cind, c in enumerate(centers)}
        result[cub] = min(dists, key=dists.get)
    return result
        

In [24]:
result = simple_voronoi(centers, weight_from_cube)
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()
img.save("simple.png")

In [32]:
class Chunk:
    def __init__(self, cid, members):
        """Each chunk has a chunk id (cid) and a """
        self.cid = cid
        self.members = members
    
    def calc_edges(self, cid_from_cube):
        """Calculates boundary, the set of cubes that border another chunk,
        self_edges, a dictionary that maps from other cids to all cubes that border that cid,
        other_edges, a dictionary that maps from other cids to all cubes in the other chunk that border this one."""
        self.boundary = set()
        self.self_edges = {}
        self.other_edges = {}
        for member in self.members:
            others = [other for other in member.neighbors() if other not in self.members and other in cid_from_cube]
            for other in others:  # Note this will skip if it's an empty list
                self.boundary.add(member)
                ocid = cid_from_cube[other]
                if cid_from_cube[other] in self.self_edges:
                    self.self_edges[ocid].add(member)
                    self.other_edges[ocid].add(other)
                else:
                    self.self_edges[ocid] = set([member])
                    self.other_edges[ocid] = set([other])

In [33]:
cids = sorted(set(result.values()))
chunks = []
for cid in cids:
    chunks.append(Chunk(cid, [k for k, v in result.items() if v == cid]))
for chunk in chunks:
    chunk.calc_edges(result)

In [38]:
[len(chunk.self_edges.keys()) for chunk in chunks]

[6,
 7,
 7,
 5,
 5,
 6,
 5,
 4,
 6,
 6,
 6,
 6,
 6,
 5,
 7,
 4,
 3,
 6,
 7,
 5,
 7,
 5,
 6,
 6,
 4,
 6,
 5,
 2,
 2,
 4,
 5,
 4,
 4,
 7,
 7,
 6,
 6,
 6,
 4,
 6,
 3,
 6,
 6,
 6,
 6,
 5,
 6,
 5,
 6,
 5,
 4,
 5,
 7,
 3,
 5,
 3,
 6,
 6,
 5,
 4,
 5,
 4,
 3,
 7,
 6,
 7,
 4,
 4,
 6,
 4,
 4,
 3,
 6,
 6,
 5,
 4,
 4,
 5,
 4,
 3]

In [59]:
triangles = set()
for cid in cids:
    for oid1 in chunks[cid].self_edges.keys():
        for oid2 in chunks[oid1].self_edges.keys():
            if cid in chunks[oid2].self_edges.keys():
                # Found a triangle!
                short_edge = min(
                    len(chunks[cid].self_edges[oid1]),  len(chunks[cid].other_edges[oid1]),
                    len(chunks[cid].self_edges[oid2]),  len(chunks[cid].other_edges[oid2]),
                    len(chunks[oid1].self_edges[oid2]), len(chunks[cid].other_edges[oid2]),
                )
                triangles.add((tuple(sorted([cid,oid1,oid2])), short_edge, sum([len(chunks[x].members) for x in [cid,oid1,oid2]])))

In [63]:
print(len(triangles))
sorted(triangles, key=lambda x: x[1], reverse=True)[:10]

162


[((5, 32, 45), 9, 296),
 ((38, 42, 49), 9, 371),
 ((4, 55, 60), 9, 263),
 ((11, 18, 43), 8, 351),
 ((12, 59, 70), 8, 318),
 ((17, 20, 23), 8, 309),
 ((58, 60, 79), 8, 251),
 ((10, 29, 66), 8, 331),
 ((12, 59, 70), 7, 318),
 ((20, 48, 74), 7, 263)]

In [61]:
quads = set()
for tri in triangles:
    for other in triangles:
        ids = set(tri[0]).union(other[0])
        if len(ids) == 4:
            quads.add((tuple(sorted(ids)), min(tri[1], other[1]), sum([len(chunks[x].members) for x in ids])))


In [62]:
print(len(quads))
sorted(quads, key=lambda x: x[1], reverse=True)[:10]

[((6, 25, 44, 51), 6, 427),
 ((17, 20, 23, 46), 6, 397),
 ((22, 25, 39, 44), 6, 396),
 ((4, 14, 22, 35), 6, 388),
 ((25, 42, 44, 51), 6, 422),
 ((6, 25, 51, 54), 6, 429),
 ((3, 6, 25, 54), 6, 449),
 ((17, 21, 23, 72), 6, 428),
 ((6, 13, 25, 51), 6, 456),
 ((21, 34, 75, 76), 6, 466)]

In [83]:
abc_ids = sorted(triangles, key=lambda x: x[1], reverse=True)[5]
print(abc_ids)


((17, 20, 23), 8, 309)


In [84]:
a,b,c = abc_ids[0]

In [85]:
abc_center = chunks[a].self_edges[b].intersection(chunks[a].self_edges[c])

In [86]:
chunks[a].self_edges[c]

{Cube(61, -73, 12),
 Cube(63, -74, 11),
 Cube(62, -74, 12),
 Cube(64, -75, 11),
 Cube(66, -76, 10),
 Cube(65, -76, 11),
 Cube(67, -77, 10),
 Cube(68, -77, 9)}

In [88]:
img = create_hex_map({cub.tuple(): rgb_from_ijk[centers[v].tuple()] for cub, v in result.items()}, max_x,max_y,n_x,n_y)
img.show()

In [76]:
abc_ids

((5, 32, 45), 9, 296)

In [78]:
CENTER_SIZE_LIST = [7,5,5,5]
KINGDOM_SIZE_LIST = [[6,4,4,4,4], [4,4,3,3], [4,4,3,3], [4,4,3]]
BORDER_SIZE_LIST = [4,4,4]

In [80]:
sum(CENTER_SIZE_LIST)+3*61+6*sum(BORDER_SIZE_LIST)

277

In [ ]:
# Basic approach here:
# hex that borders all three kingdoms should be the middle of a seven-hex. At least one of them should be--maybe our first guess is wrong.
# Make that the central duchy of abc. Then we have to get 3 size-5 duchies; might as well take one from a, b, and c.
# Now we need ab/bc/ac; those can start from the edge zones and grow outward from there.

abc0 = Chunk()